In [ ]:
import sys
import time
import numpy as np
import tensorflow as tf
import cv2
import os
import glob
import math
import shutil

from IPython.display import clear_output, display, Image
from joblib import Parallel, delayed

sys.path.append('../')
from model import Model

sys.path.append("../face_detection/")
from utils import label_map_util
from utils import visualization_utils_color as vis_util
from tensorflow_face_detector import TensoflowFaceDector

In [ ]:
names = {
    0: "iguchi",
    1: "ushio",
    2: "kakizaki",
    3: "kageyama",
    4: "kato",
    5: "saito",
    6: "sasaki_k",
    7: "sasaki_m",
    8: "takase" ,
    9: "takamoto" ,
    10: "higashimura",
    11: "kanemura",
    12: "kawata",
    13: "kosaka",
    14: "tomita",
    15: "nibu",
    16: "hamagishi",
    17: "matsuda",
    18: "miyata",
    19: "watanabe",
    20: "kamimura",
}

In [ ]:
def detect_n_faces(imageFile):
    image = cv2.imread(imageFile)
    faceDetectImage = image.copy()
    [h, w] = faceDetectImage.shape[:2]

    (boxes, scores, classes, num_detections) = tDetector.run(faceDetectImage)

    faceBoxes = vis_util.visualize_boxes_and_labels_on_image_array(
        faceDetectImage,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=4)
    
    personCount = len(faceBoxes)
        
    imageHeight, imageWidth = image.shape[:2]
    
    originBoxes = []
    cropBoxes = []
    
    for index, box in enumerate(faceBoxes):
        ymin, xmin, ymax, xmax = box
        (left, right, top, bottom) = (int(xmin * imageWidth), int(xmax * imageWidth), int(ymin * imageHeight), int(ymax * imageHeight))
        originBoxes.append((left, right, top, bottom))  # TensorFlowの顔検出で切り取った領域
        
        cropWidth = right - left
        cropHeight = bottom - top
        
        # 長辺に合わせる
        if cropHeight > cropWidth:
            diff  = (cropHeight - cropWidth) / 2
            if int(left - diff) < 0 or int(right + diff) > imageWidth:
                top = int(top + diff)
                bottom = int(bottom - diff)
            else:
                left = int(left - diff)
                right = int(right + diff)
        else:
            diff = (cropWidth - cropHeight) / 2
            if int(top - diff) < 0 or int(bottom + diff) > imageHeight:
                left = int(left + diff)
                right = int(right - diff)
            else:
                top = int(top - diff)
                bottom = int(bottom + diff)
        
        cropBoxes.append((left, right, top, bottom))   # 顔検出に用いる正方形領域
        
    return originBoxes, cropBoxes

In [ ]:
# データ解析
def read_image(images):     
    image = tf.cast(images, tf.float32)
    image = image / 255  # 画像データを、0～1の範囲に変換する
    image = tf.reshape(image, [128, 128, 3])
     
    return image

In [ ]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '../face_detection/model/frozen_inference_graph_face.pb'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '../face_detection/protos/face_label_map.pbtxt'
NUM_CLASSES = 2

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

tDetector = TensoflowFaceDector(PATH_TO_CKPT)# モデルのロード

In [ ]:
# 出力フォルダ作成
originImagePath = '../images/origin/'
originImageMemberPath = glob.glob(originImagePath + "*")
for index, memberPath in enumerate(originImageMemberPath):
    memberPath = memberPath.replace("\\", '/') + '/'
    originImageMemberPath[index] = memberPath        
    outputOriginImageMemberPath = 'output_images/' + 'origin/' + memberPath.split('/')[-2] + '/'
    os.makedirs(outputOriginImageMemberPath + '1/', exist_ok=True)
    os.makedirs(outputOriginImageMemberPath + '2/', exist_ok=True)
    os.makedirs(outputOriginImageMemberPath + '3_over/', exist_ok=True)

os.makedirs('output_images/origin/others/1/', exist_ok=True)
os.makedirs('output_images/origin/others/2/', exist_ok=True)
os.makedirs('output_images/origin/others/3_over/', exist_ok=True)

In [ ]:
def faceIdentificationAndCopyFiles(imageFiles, maxFaces, ckptPath):
    
    fileNames = [] # ファイル名のリスト
    detectFacesNum = [] # ファイルごとの検出した顔の数
    cropImages = [] # 切り取った顔のリスト
    
    for imageFile in imageFiles:
        sourceImage = cv2.imread(imageFile)
        originBoxes, cropBoxes = detect_n_faces(imageFile)

        # 指定人数より多い or 1つも顔が検出されなければ次へ
        if len(cropBoxes) > maxFaces or len(cropBoxes) == 0:
            continue

        fileNames.append(imageFile)
        
        # 検出された顔の数をカウント
        nDetectFace = 0
        for box in cropBoxes:
            (left, right, top, bottom) = box
            cropImages.append(sourceImage[top:bottom, left:right])
            nDetectFace += 1
            
        detectFacesNum.append(nDetectFace)

    # テンソル用の準備
    imgs = np.empty((0, 128, 128, 3)) #empty dummy array, we will append to this array all the images
    for img in cropImages:
        img = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_LANCZOS4)
        img = img[:, :, ::-1].copy()
        imgs = np.append(imgs, np.array(img).reshape((1, 128, 128, 3)), axis=0)

    with tf.Graph().as_default():
        dataset = tf.data.Dataset.from_tensor_slices(imgs)
        dataset = dataset.map(read_image, len(cropImages))  # データの形式、形状を変更
        dataset = dataset.batch(len(cropImages))  # 連続するレコードをバッチに結合
        iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)  # イテレータを作成
        images = iterator.get_next()  # イテレータの次の要素を取得

        initOp = iterator.make_initializer(dataset)  # イテレータを初期化

        model = Model(images, 128, 21, len(cropImages), trainable=False)

        with tf.Session() as session:
            session.run(initOp)  # データの初期化
            saver = tf.train.Saver()
            saver.restore(session, ckptPath)

            estimationResult, resultImages = session.run([model.logits, images], feed_dict={model.keepProb: 1.0, model.isTraining: False})
            
    for index, fileName in enumerate(fileNames):
        # 2人以下ならそのままフォルダ名に
        if detectFacesNum[index] < 3:
            saveFolderDetectFaces = str(detectFacesNum[index]) + '/'
        else:
            saveFolderDetectFaces = '3_over/'
        
        # 検出した顔を識別してフォルダにコピー
        for faceIndex in range(detectFacesNum[index]):
            estimation = estimationResult[index + faceIndex]               
            
            if np.max(estimation) < 0.5:
                saveFolder = 'output_images/' + 'origin/' + 'others/' + saveFolderDetectFaces
            else:
                idFaceMemberName = names[np.argmax(estimation)]
                saveFolder = 'output_images/'+ 'origin/' + idFaceMemberName + '/' + saveFolderDetectFaces
            shutil.copy(fileName, saveFolder)

In [ ]:
nPickImages = 50
ckptPath = 'recognition_model/model.ckpt-101881'
for memberImageFolder in originImageMemberPath:
    imageFiles = glob.glob(memberImageFolder + '*')
    nFiles = len(imageFiles)
    
    for i in range(math.ceil(nFiles / nPickImages)):
        startIndex = i * nPickImages
        if (i + 1) * nPickImages > nFiles:
            endIndex = nFiles
        else:
            endIndex = (i + 1) * nPickImages
            
        pickedImages = imageFiles[startIndex : endIndex]
        
        faceIdentificationAndCopyFiles(pickedImages, 2, ckptPath)